### Import Libraries

In [2]:
import pandas as pd

### Load Dataset

In [ ]:
dataset = pd.read_csv('hasil_scraping.csv')
dataset.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,be6c0b77-7c2c-45aa-99be-ecf34ded00d0,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Lots of problems. If you use a template and cu...,1,4216,4.1.1007.2,2024-11-24 19:47:40,Hey! We are sorry for the experience you have ...,2024-11-24 12:27:44,4.1.1007.2
1,f64d3b75-3d94-491f-9b66-22a3079d488e,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,LinkedIn has been a game-changer for my career...,5,1,4.1.1052,2025-04-02 10:38:56,NaN,NaN,4.1.1052
2,6514bc74-a84c-491e-ad33-a9b5dccddded,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"LinkedIn's strength lies in its vast network, ...",5,31,4.1.1050,2025-03-31 23:38:25,NaN,NaN,4.1.1050
3,234d2b20-2f9d-45bc-acd6-1113a59e1677,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,For the past few months I haven't been able to...,1,455,4.1.1048.1,2025-03-24 20:24:29,NaN,NaN,4.1.1048.1
4,c6f53ce1-ec57-49ae-b8dc-01a45c88302d,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,I am having major problems with resetting my p...,1,54,4.1.1045.1,2025-03-18 01:00:22,NaN,NaN,4.1.1045.1


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              90000 non-null  object
 1   userName              90000 non-null  object
 2   userImage             90000 non-null  object
 3   content               89998 non-null  object
 4   score                 90000 non-null  int64 
 5   thumbsUpCount         90000 non-null  int64 
 6   reviewCreatedVersion  75376 non-null  object
 7   at                    90000 non-null  object
 8   replyContent          34162 non-null  object
 9   repliedAt             34162 non-null  object
 10  appVersion            75376 non-null  object
dtypes: int64(2), object(9)
memory usage: 7.6+ MB


### Cleaning Dataset

In [4]:
clean_dataset = dataset.dropna()

In [6]:
clean_dataset = clean_dataset.drop_duplicates()

In [7]:
clean_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28513 entries, 0 to 89994
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              28513 non-null  object
 1   userName              28513 non-null  object
 2   userImage             28513 non-null  object
 3   content               28513 non-null  object
 4   score                 28513 non-null  int64 
 5   thumbsUpCount         28513 non-null  int64 
 6   reviewCreatedVersion  28513 non-null  object
 7   at                    28513 non-null  object
 8   replyContent          28513 non-null  object
 9   repliedAt             28513 non-null  object
 10  appVersion            28513 non-null  object
dtypes: int64(2), object(9)
memory usage: 2.6+ MB
